# **AI LAB 2023**
### A. Calderoni, A. Fabbricatore, S. Mancini, A. Pranzo, B. Zanini

## Experimental Background
Roba da scrivere

## Exploratory Data Analysis and Cleaning
Lavorate sulla vostra versione di questo, poi ci mettiamo insieme e completiamo qui mettendo insieme le idee

## Filtering and Normalisation
Come sopra

## Dimensionality Reduction
Facciamo insieme

## Model
Facciamo insieme